
# Laboratório: DW de Saúde Pública — COVID‑19 (Brasil)

**Tema:** Saúde Pública (COVID‑19)  
**Bases:**  
1) **OWID COVID** (casos/óbitos por país) — CSV público  
2) **OWID Vaccinations** (vacinação por país) — CSV público  
3) **BrasilAPI Feriados** (enriquecimento de `dim_date` com `is_holiday`)

> Objetivo: consolidar dados diários de COVID e vacinação para o Brasil em um **DW (Star Schema)**, realizar **ETL** com pandas e aplicar **clustering** para segmentar períodos epidemiológicos.
